## PyTorch 中的数据读取

### Dataset 类
- __init__()：构造函数，可自定义数据读取方法以及进行数据预处理；
- __len__()：返回数据集大小；
- __getitem__()：索引数据集中的某一个数据。

In [1]:
import torch
from torch.utils.data import Dataset

# 使用 Dataset 类定义一个 Tensor 类型的数据集。
class MyDataset(Dataset):
    # 构造函数
    def __init__(self, data_tensor, target_tensor):
        self.data_tensor = data_tensor
        self.target_tensor = target_tensor

    # 返回数据集大小
    def __len__(self):
        return self.data_tensor.size(0)
    
    # 返回索引的数据与标签
    def __getitem__(self, index):
        return self.data_tensor[index], self.target_tensor[index]

In [2]:
# 生成数据
data_tensor = torch.randn(10,3)
target_tensor = torch.randint(2, (10, ))
data_tensor, target_tensor

(tensor([[ 0.3228,  0.1697, -0.6006],
         [-0.3494,  0.1198,  0.2255],
         [ 0.1142,  1.3729, -0.0560],
         [ 0.0198,  1.1763,  1.2440],
         [ 0.0955, -0.6867, -1.5766],
         [ 1.4895,  0.6918, -0.6449],
         [ 0.8946, -1.7621,  1.9322],
         [ 0.3471,  0.2476,  0.6429],
         [-0.2112, -0.6215, -0.1333],
         [ 0.0729, -1.0786,  1.1755]]),
 tensor([1, 1, 0, 0, 1, 0, 0, 0, 1, 1]))

In [3]:
# 将数据封装成Dataset
my_dataset = MyDataset(data_tensor, target_tensor)
my_dataset

In [4]:
# 查看数据集大小
len(my_dataset)

10

In [5]:
# 使用索引调用数据
my_dataset[0]

(tensor([ 0.3228,  0.1697, -0.6006]), tensor(1))

### DataLoader 类
在实际项目中，如果数据量很大，考虑到内存有限、I/O 速度等问题，在训练过程中不可能一次性的将所有数据全部加载到内存中，也不能只用一个进程去加载，所以就需要多进程、迭代加载，而 DataLoader 就是基于这些需要被设计出来的。

DataLoader 是一个迭代器，最基本的使用方法就是传入一个 Dataset 对象，它会根据参数 batch_size 的值生成一个 batch 的数据，节省内存的同时，它还可以实现多进程、数据打乱等处理。

In [6]:
from torch.utils.data import DataLoader

tensor_dataloader = DataLoader(dataset=my_dataset, # 传入的数据集，必须参数
                               batch_size=2,       # 输出的batch大小
                               shuffle=True,       # 数据是否打乱
                               num_workers=0)      # 进程数，0表示只有主进程

tensor_dataloader

- dataset：Dataset 类型，输入的数据集，必须参数；
- batch_size：int 类型，每个 batch 有多少个样本；
- shuffle：bool 类型，在每个 epoch 开始的时候，是否对数据进行重新打乱；
- num_workers：int 类型，加载数据的进程数，0 意味着所有的数据都会被加载进主进程，默认为 0。

In [7]:
# 以循环形式输出
for data, target in tensor_dataloader:
    print(data, target)

tensor([[ 0.3228,  0.1697, -0.6006],
        [ 0.8946, -1.7621,  1.9322]]) tensor([1, 0])
tensor([[-0.3494,  0.1198,  0.2255],
        [ 0.1142,  1.3729, -0.0560]]) tensor([1, 0])
tensor([[ 0.3471,  0.2476,  0.6429],
        [-0.2112, -0.6215, -0.1333]]) tensor([0, 1])
tensor([[ 0.0955, -0.6867, -1.5766],
        [ 0.0729, -1.0786,  1.1755]]) tensor([1, 1])
tensor([[ 1.4895,  0.6918, -0.6449],
        [ 0.0198,  1.1763,  1.2440]]) tensor([0, 0])


In [11]:
# 输出一个batch
print(f'One batch tensor data: {iter(tensor_dataloader).__next__()}')

One batch tensor data: [tensor([[ 0.1142,  1.3729, -0.0560],
        [ 0.0729, -1.0786,  1.1755]]), tensor([0, 1])]


## 什么是 Torchvision
Torchvision 是一个和 PyTorch 配合使用的 Python 包。它不只提供了一些常用数据集，还提供了几个已经搭建好的经典网络模型，以及集成了一些图像数据处理方面的工具，主要供数据预处理阶段使用。<br>
简单地说，Torchvision 库就是 *<b>常用数据集 + 常见网络模型 + 常用图像处理方法。<b>*

### 利用 Torchvision 读取数据

Torchvision 库中的torchvision.datasets包中提供了丰富的图像数据集的接口。常用的图像数据集，例如 MNIST、COCO 等，这个模块都为我们做了相应的封装。

![](./torchvision_datasets.png)

## MNIST 数据集简介
MNIST 数据集是一个著名的手写数字数据集，因为上手简单，在深度学习领域，手写数字识别是一个很经典的学习入门样例。

### 数据读取

In [1]:
import torchvision

mnist_dataset = torchvision.datasets.MNIST(root='./data',
                                           train=True,
                                           transform=None,
                                           target_transform=None,
                                           download=True)

c:\Envs\transformers\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


100%|██████████| 9912422/9912422 [00:04<00:00, 2233555.67it/s]


Extracting ./data\MNIST\raw\train-images-idx3-ubyte.gz to ./data\MNIST\raw



100%|██████████| 28881/28881 [00:00<00:00, 86662.79it/s]


Extracting ./data\MNIST\raw\train-labels-idx1-ubyte.gz to ./data\MNIST\raw



100%|██████████| 1648877/1648877 [00:02<00:00, 793486.99it/s] 


Extracting ./data\MNIST\raw\t10k-images-idx3-ubyte.gz to ./data\MNIST\raw



100%|██████████| 4542/4542 [00:00<?, ?it/s]

Extracting ./data\MNIST\raw\t10k-labels-idx1-ubyte.gz to ./data\MNIST\raw



- root：是一个字符串，用于指定你想要保存 MNIST 数据集的位置。如果 download 是 Flase，则会从目标位置读取数据集；
- download：是布尔类型，表示是否下载数据集。如果为 True，则会自动从网上下载这个数据集，存储到 root 指定的位置。如果指定位置已经存在数据集文件，则不会重复下载；
- train：是布尔类型，表示是否加载训练集数据。如果为 True，则只加载训练数据。如果为 False，则只加载测试数据集。这里需要注意，并不是所有的数据集都做了训练集和测试集的划分，这个参数并不一定是有效参数，具体需要参考官方接口说明文档；
- transform：用于对图像进行预处理操作，例如数据增强、归一化、旋转或缩放等。这些操作我们会在下节课展开讲解；
- target_transform：用于对图像标签进行预处理操作。

In [3]:
mnist_dataset, type(mnist_dataset)

(Dataset MNIST
     Number of datapoints: 60000
     Root location: ./data
     Split: Train,
 torchvision.datasets.mnist.MNIST)

### 数据预览

In [4]:
mnist_dataset_list = list(mnist_dataset)
mnist_dataset_list

[(<PIL.Image.Image image mode=L size=28x28>, 5),
 (<PIL.Image.Image image mode=L size=28x28>, 0),
 (<PIL.Image.Image image mode=L size=28x28>, 4),
 (<PIL.Image.Image image mode=L size=28x28>, 1),
 (<PIL.Image.Image image mode=L size=28x28>, 9),
 (<PIL.Image.Image image mode=L size=28x28>, 2),
 (<PIL.Image.Image image mode=L size=28x28>, 1),
 (<PIL.Image.Image image mode=L size=28x28>, 3),
 (<PIL.Image.Image image mode=L size=28x28>, 1),
 (<PIL.Image.Image image mode=L size=28x28>, 4),
 (<PIL.Image.Image image mode=L size=28x28>, 3),
 (<PIL.Image.Image image mode=L size=28x28>, 5),
 (<PIL.Image.Image image mode=L size=28x28>, 3),
 (<PIL.Image.Image image mode=L size=28x28>, 6),
 (<PIL.Image.Image image mode=L size=28x28>, 1),
 (<PIL.Image.Image image mode=L size=28x28>, 7),
 (<PIL.Image.Image image mode=L size=28x28>, 2),
 (<PIL.Image.Image image mode=L size=28x28>, 8),
 (<PIL.Image.Image image mode=L size=28x28>, 6),
 (<PIL.Image.Image image mode=L size=28x28>, 9),
 (<PIL.Image.Image i

In [5]:
display(mnist_dataset_list[0][0])
print(f'Image label is: {mnist_dataset_list[0][1]}')

Image label is: 5
